In [1]:
import sys
import os

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cm

%matplotlib inline

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from pandas import plotting

AEM_database_train_raw = pd.read_csv(r'AEM_database_train.csv')

# Database pre-processing = Section "Preprocessing and Feature Selection"
# Train data pre-processing
mordred_descriptor_train = AEM_database_train_raw.loc[:,'ABC_A':]
mordred_descriptor_train = mordred_descriptor_train.dropna(how='any', axis=1)

mordred_descriptor_train.isnull().any(axis=0)
mordred_descriptor_train.isnull().sum()

nunique = mordred_descriptor_train.apply(pd.Series.nunique)
cols_to_drop = nunique[nunique == 1].index
mordred_descriptor_train = mordred_descriptor_train.drop(cols_to_drop, axis = 1)

polymer_info = AEM_database_train_raw.loc[:, "Membrane Name (abbreviation)":"Membrane Anion Conductivity [S/cm]"]
AEM_database_train = pd.concat([polymer_info, mordred_descriptor_train], axis=1)

AEM_database_train = AEM_database_train.reset_index()
AEM_database_train = AEM_database_train.drop(['index'], axis = 1)

# Finalizing train database for PCA
train_polymer_ratio = AEM_database_train.loc[:, ["Block A (x)", "Block B (1-x)", "Block A/B ratio","Polymer type"]]
train_polymer_ratio = train_polymer_ratio.reset_index()
train_polymer_ratio = train_polymer_ratio.drop(['index'], axis = 1)
PCA_train_A = AEM_database_train.loc[:, "ABC_A":"mZagreb2_A"]
PCA_train_B = AEM_database_train.loc[:, "ABC_B":]

# Test data pre-processing
AEM_database_test_raw = pd.read_csv(r'AEM_database_test.csv')
column_names = list(AEM_database_train.columns)
AEM_database_test = AEM_database_test_raw.filter(items=column_names)

# Finalizing test database for PCA
test_polymer_ratio = AEM_database_test.loc[:, ["Block A (x)", "Block B (1-x)", "Block A/B ratio","Polymer type"]]
test_polymer_ratio = test_polymer_ratio.reset_index()
test_polymer_ratio = test_polymer_ratio.drop(['index'], axis = 1)
PCA_test_A = AEM_database_test.loc[:, "ABC_A":"mZagreb2_A"]
PCA_test_B = AEM_database_test.loc[:, "ABC_B":]

In [ ]:
# Carrying out PCA = First half of Section "Building Unsupervised Machine Learning Models and Cluster Analysis"

# PCA of Block A
data_A = PCA_train_A.loc[:,:].values

# Standardize the data
scaler_A = StandardScaler().fit(data_A)
data_A = scaler_A.transform(data_A)

# Create a PCA object with 32 components
pca_A = PCA(n_components=32, random_state=0)

# Fit the PCA model to the data
principalcomponents_A = pca_A.fit_transform(data_A)

# Get the weight of each component
pcaweight_A = pca_A.components_
pcaweight_A = pd.DataFrame(data=pcaweight_A)

# Get train DataFrame with principal components
train_principalDf_A = pd.DataFrame(data=principalcomponents_A, columns=[f'principal component {i}_A' for i in range(32)])

# Contribution Ratio for train database
train_pca_cont_A = pd.DataFrame(pca_A.explained_variance_ratio_, index=["PC{}".format(x + 1) for x in range(len(train_principalDf_A.columns))])

# Fitting of same model to test data, to ensure that the same equation with train data is applied
test_A_PCA = scaler_A.transform(PCA_test_A)
test_A_PCA = pca_A.transform(test_A_PCA)
test_principalDf_A = pd.DataFrame(data=test_A_PCA, columns=[f'principal component {i}_A' for i in range(32)])

# Plot train data cumulative contribution rate graph for block A
import matplotlib.ticker as ticker
plt.figure(figsize=(20,20))
plt.gca().get_xaxis().set_major_locator(ticker.MaxNLocator(integer=True))
plt.plot([0] + list( np.cumsum(pca_A.explained_variance_ratio_)), "-o", markersize=13, linewidth=4, color='navy')
plt.xlabel("Number of Principal Components (PC)", fontsize=46)
plt.ylabel("Cumulative Contribution Rate", fontsize=46)
plt.grid()
plt.xticks(fontsize=40)
plt.yticks(fontsize=40)

# Export graph to png if needed
#plt.savefig('contribution_weight_A.png')

# PCA of Block B
data_B = PCA_train_B.loc[:,:].values

# Standardize the data
scaler_B = StandardScaler().fit(data_B)
data_B = scaler_B.transform(data_B)

# Create a PCA object with 32 components
pca_B = PCA(n_components=32, random_state=0)

# Fit the PCA model to the data
principalcomponents_B = pca_B.fit_transform(data_B)

# Get the weight of each component
pcaweight_B = pca_B.components_
pcaweight_B = pd.DataFrame(data=pcaweight_B)

# Get train DataFrame with principal components
train_principalDf_B = pd.DataFrame(data=principalcomponents_B, columns=[f'principal component {i}_B' for i in range(32)])

# Contribution Ratio for train database
train_pca_cont_B = pd.DataFrame(pca_B.explained_variance_ratio_, index=["PC{}".format(x + 1) for x in range(len(train_principalDf_B.columns))])

# Fitting of same model to test data, to ensure that the same equation with train data is applied
test_B_PCA = scaler_B.transform(PCA_test_B)
test_B_PCA = pca_B.transform(test_B_PCA)
test_principalDf_B = pd.DataFrame(data=test_B_PCA, columns=[f'principal component {i}_B' for i in range(32)])

# Plot train data cumulative contribution rate graph for block A
import matplotlib.ticker as ticker
plt.figure(figsize=(20,20))
plt.gca().get_xaxis().set_major_locator(ticker.MaxNLocator(integer=True))
plt.plot([0] + list( np.cumsum(pca_B.explained_variance_ratio_)), "-o", markersize=13, linewidth=4, color='royalblue')
plt.xlabel("Number of Principal Components (PC)", fontsize=46)
plt.ylabel("Cumulative Contribution Rate", fontsize=46)
plt.grid()
plt.xticks(fontsize=40)
plt.yticks(fontsize=40)

# Export graph to png if needed
#plt.savefig('contribution_weight_B.png')

#### End of PCA model

# Pre-processing of data obtained from PCA for use in UMAP

train_principalDf_combined = pd.concat([train_principalDf_A,train_principalDf_B], axis = 1)
train_principalDf_combined = train_principalDf_combined.reset_index()
train_principalDf_combined = train_principalDf_combined.drop(['index'], axis = 1)

test_principalDf_combined = pd.concat([test_principalDf_A,test_principalDf_B], axis = 1)
test_principalDf_combined = test_principalDf_combined.reset_index()
test_principalDf_combined = test_principalDf_combined.drop(['index'], axis = 1)

database_UMAP_train = pd.concat([train_polymer_ratio, train_principalDf_combined], axis = 1)

database_UMAP_test = pd.concat([test_polymer_ratio, test_principalDf_combined], axis = 1)

In [ ]:
# Carrying out UMAP = Second half of Section "Building Unsupervised Machine Learning Models and Cluster Analysis"

from umap import UMAP

### UMAP for train database
# Create a UMAP object and fit
umap = UMAP(random_state=80, n_neighbors=15, n_components=2).fit(database_UMAP_train)

# Get the UMAP embedding for train data
embedding = umap.transform(database_UMAP_train)

# Plot the UMAP embedding for train data
y = AEM_database_train.loc[:, "Membrane Anion Conductivity [S/cm]"]

plt.figure(figsize=(20,20))

scatter = plt.scatter(embedding[:, 0], embedding[:, 1],c=y, cmap=cm.tab20b, s=200, marker='o', edgecolors='pink')
cb = plt.colorbar(scatter).set_label('Membrane Anion Conductivity / S cm$^{-1}$', size=46)

#tick label size of graph
scatter.figure.axes[0].tick_params(axis="both", labelsize = 40)
#tick label size of colorbar
scatter.figure.axes[1].tick_params(axis="y", labelsize = 40)
plt.show()
#plt.savefig('UMAP_train.png')

### UMAP for test database
# Get the UMAP embedding for test data
embedding_test = umap.transform(database_UMAP_test)

# Plot the UMAP embedding for train data with test data overlay
y = AEM_database_train.loc[:, "Membrane Anion Conductivity [S/cm]"]

# Color code for test polymers
test_polymer_color_code = AEM_database_test.replace(to_replace = ['PPO-Pip','PPO-OPip','PPO-PipOH','PAEK-HQACz-0.5','PAEK-HQACz-0.6','PAEK-HQACz-0.7','PTPipQ100','PTPipQ83','PpTASU','PpTDMP','PmTASU','PmTDMP','F20C9N','H22C9N'],value = ['1','1','1','2','2','2','3','3','3','3','3','3','4','4'])
z = test_polymer_color_code.loc[:, 'Membrane Name (abbreviation)']
z = pd.to_numeric(z)

plt.figure(figsize=(20,20))

scatter = plt.scatter(embedding[:, 0], embedding[:, 1],c=y, cmap=cm.tab20b, s=250, marker='o', edgecolors='pink', linewidths=1)
cb = plt.colorbar(scatter).set_label('Membrane Anion Conductivity / S cm$^{-1}$', size=46)

scatter = plt.scatter(embedding_test[:, 0], embedding_test[:, 1],c=z, cmap=cm.tab10, s=300, marker='o', edgecolors='black', linewidths=3)

#tick label size of graph
scatter.figure.axes[0].tick_params(axis="both", labelsize = 40)
#tick label size of colorbar
scatter.figure.axes[1].tick_params(axis="y", labelsize = 40)

plt.show()
#plt.savefig('UMAP_train+test.png')